In [7]:
using Plots;
gadfly();

In [8]:
include("../fdtd/update.jl");
include("../fdtd/sources.jl");
include("../fdtd/boundaries.jl");
using update;
using sources;

In [9]:
#Global parameters
size = 200
endTime = 700
num_snaps = 350
snap_step = div(endTime, num_snaps)
#Grid
hy = zeros(size);
ez = zeros(size);
# output params
ez_snapshot = Array{Any}(num_snaps);
hy_snapshot = Array{Any}(num_snaps);

fieldsum = 1;
minsumtime = 1;
minfieldsnap = zeros(size);

In [10]:
#Time steps

for time in 1:endTime
    #Field update equations
    
    update.update_magnetic_field!(ez, hy)
    update.update_electric_field!(ez, hy)
    # Source at center
    sources.additive_gaussian_source!(ez, div(length(ez),2), time)

    #Boundaries
    boundaries.trivial_abc!(ez, hy, 200)
    boundaries.pec_boundary!(ez, 1)
    
    #Catch minimum E-Field
    if time > 50
        if fieldsum > sum(ez.^2)
            minsumtime = time
            fieldsum = sum(ez.^2)
            minfieldsnap = copy(ez)
        end
    end
    
    #Snapshots for animation
    if mod(time, snap_step) == 0
        ez_snapshot[div(time,snap_step)] = (time, copy(ez))
        hy_snapshot[div(time,snap_step)] = (time, copy(hy).*globals.imp0)        
    end
    
end
print(minsumtime, '\n')
print(fieldsum)

400
1.2182481907510104e-62

In [13]:
anim = Animation()

for i = 1:num_snaps
    p = plot(1:size, ez_snapshot[i][2], lab="Ez")
    plot!(1:size, hy_snapshot[i][2], lab="Hy*imp0")
    
    time = ez_snapshot[i][1]
    plot!(ann=[(150, 1.5, "time =$time")])
    plot!(ann=[(0, 1.1, "PEC")])
    plot!(ann=[(180, 1.1, "ABC")])    
    plot!(xlims=(1, 200), ylims=(-2, 2))
    frame(anim, p)
end
gif(anim, "./PEC-ABC.gif", fps=15)

INFO: Saved animation to /media/storage/Documents/Github/1d-fdtd/examples/PEC-ABC.gif


Plots.AnimatedGif("/media/storage/Documents/Github/1d-fdtd/examples/PEC-ABC.gif")